# Databento symbology

**Welcome to the Databento client library symbology tutorial!**

We'll walk through how to request and work with symbology mappings, and how we can apply these to time series data.

**Note:** 

For information on our symbology, refer to https://docs.databento.com/reference-historical/basics/symbology. 

For a more detailed API reference, refer to https://docs.databento.com/reference-historical.

This tutorial covers the following:
- Using the historical client to request symbology mappings
- Using the mappings to add a native symbol column to time series data

**Tip:** You can call help() on any class or method to see the 'docstring'.

## Installation and setup

Firtly, ensure you have the latest `databento` client library installed:
```bash
pip install -U databento
```

## Historical data client

Once you've installed the Python client library, you can import it and initialize a historical client for requests. We'll use this `client` throughout the rest of the tutorial.

To initialize a client, you need to provide a valid API key. You can find these on the API Keys page of your Databento portal at https://databento.com.

In [33]:
import databento as db

client = db.Historical(key="YOUR_API_KEY")

## Requesting time series data

Firstly, we'll request `trades` time series data for CME Globex MDP 3.0 dataset, for some highly liquid futures outrights.

In [21]:
data = client.timeseries.stream(
    dataset="GLBX.MDP3",
    symbols="ESH1,GEH1,CLH1,6BH1",
    schema="trades",
    start="2020-12-27",
    end="2020-12-30",
    limit=1000,  # <-- request limited to 1000 records
)

## Requesting symbology resolution

If we'd like to know the native symbol for each trade, we'll have to request to resolve the mappings from the native symbology we queried to the product IDs contained in the data records.
In the near future it will be possible to specify an `stype_out` of **native**, which will append the native symbol to every data record.

In [25]:
response = client.symbology.resolve(
    dataset="GLBX.MDP3",
    symbols="ESH1,GEH1,CLH1,6BH1",
    stype_in="native",
    stype_out="product_id",
    start_date="2020-12-27",
    end_date="2020-12-30",
)

response

{'result': {'ESH1': [{'d0': '2020-12-27', 'd1': '2020-12-30', 's': '5482'}],
  'GEH1': [{'d0': '2020-12-27', 'd1': '2020-12-30', 's': '895045'}],
  'CLH1': [{'d0': '2020-12-27', 'd1': '2020-12-30', 's': '169953'}],
  '6BH1': [{'d0': '2020-12-27', 'd1': '2020-12-30', 's': '8382'}]},
 'symbols': ['ESH1', 'GEH1', 'CLH1', '6BH1'],
 'stype_in': 'native',
 'stype_out': 'product_id',
 'start_date': '2020-12-27',
 'end_date': '2020-12-30',
 'partial': [],
 'not_found': [],
 'message': 'OK',
 'status': 0}

## Using symbology mappings

Now that we have some symbology mappings we can use these to create a column of native symbols to help us identify the product for each trade.

In [28]:
mappings = response["result"]
mappings

{'ESH1': [{'d0': '2020-12-27', 'd1': '2020-12-30', 's': '5482'}],
 'GEH1': [{'d0': '2020-12-27', 'd1': '2020-12-30', 's': '895045'}],
 'CLH1': [{'d0': '2020-12-27', 'd1': '2020-12-30', 's': '169953'}],
 '6BH1': [{'d0': '2020-12-27', 'd1': '2020-12-30', 's': '8382'}]}

Given there is only one mapping for each product for the entire time range of three days. Lets create a simple index of `product_id` to `native` symbol.

In [31]:
index = {int(v[0]["s"]): k for k, v in mappings.items()}
index

{5482: 'ESH1', 895045: 'GEH1', 169953: 'CLH1', 8382: '6BH1'}

In [ ]:
data["symbol"] = index[data["product_id"]]